## Question 1. [IPO] Withdrawn IPOs by Company Type

In [1]:
!pip install yfinance


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import requests
import re

import yfinance as yf
import pandas_datareader as pdr

import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

import matplotlib.pyplot as plt

In [3]:
from io import StringIO

def get_withdrawn_ipos(url: str) -> pd.DataFrame:
    """
    Fetch and return HTML tables from the given URL using pandas.
    Returns the first table if multiple are found.
    """
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found on the page {url}.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed for {url}: {e}")
    except ValueError as ve:
        print(f"Data error for {url}: {ve}")
    except Exception as ex:
        print(f"An unexpected error occurred for {url}: {ex}")

    return pd.DataFrame()

In [4]:
withdrawn_ipos_df = get_withdrawn_ipos("https://stockanalysis.com/ipos/withdrawn/")

In [5]:
len(withdrawn_ipos_df)

100

In [6]:
withdrawn_ipos_df.head()

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-


In [7]:
withdrawn_ipos_df.columns

Index(['Symbol', 'Company Name', 'Price Range', 'Shares Offered'], dtype='object')

In [8]:
def categorize_company_class(df: pd.DataFrame) -> pd.DataFrame:
    if 'Company Name' not in df.columns:
        print("Error: 'Company Name' column not found in the DataFrame.")
        return pd.DataFrame()

    pattern_order = [
        ('Corp', ['acquisition corp', 'acquisition corporation']),
        ('Inc', ['inc', 'incorporated']),
        ('Group', ['group']),
        ('Limited', ['ltd', 'limited']),
        ('Holdings', ['holdings']),
    ]

    def get_company_class(company_name: str) -> str:
        if not isinstance(company_name, str):
            return "Other"

        lower_name = company_name.lower()

        for category, keywords in pattern_order:
            for keyword in keywords:
                if keyword in lower_name:
                    return category
        return "Other"

    df['Company Class'] = df['Company Name'].apply(get_company_class)
    return df

In [9]:
categorize_company_class(withdrawn_ipos_df)

,Symbol,Company Name,Price Range,Shares Offered,Company Class
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc
2,AURN,"Aurion Biotech, Inc.",-,-,Inc
3,ROTR,"PHI Group, Inc.",-,-,Inc
4,ONE,One Power Company,-,-,Other
...,...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-,Inc
96,CHO,Chobani Inc.,-,-,Inc
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc
98,GLGX,"Gerson Lehrman Group, Inc.",-,-,Inc


In [10]:
withdrawn_ipos_df.groupby('Company Class').size()

Company Class
Corp        21
Group        4
Holdings     1
Inc         51
Limited     17
Other        6
dtype: int64

In [11]:
def calculate_average_price(df: pd.DataFrame) -> pd.DataFrame:
    if 'Price Range' not in df.columns:
        print("Error: 'Price Range' column not found in the DataFrame.")
        return pd.DataFrame()

    def parse_price_range(price_range_str: str):
        """Helper function to parse a single price range string."""
        if not isinstance(price_range_str, str):
            return None

        cleaned_str = price_range_str.replace('$', '').strip()

        if cleaned_str == '-':
            return None
        elif '-' in cleaned_str:
            try:
                lower_price, upper_price = map(float, cleaned_str.split('-'))
                return (lower_price + upper_price) / 2.0
            except ValueError:
                print(f"Warning: Could not parse price range '{price_range_str}'. Returning None.")
                return None
        else:
            try:
                return float(cleaned_str)
            except ValueError:
                print(f"Warning: Could not parse single price '{price_range_str}'. Returning None.")
                return None

    df['Avg. Price'] = df['Price Range'].apply(parse_price_range)

    return df

In [12]:
calculate_average_price(withdrawn_ipos_df)

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. Price
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc,3.5
2,AURN,"Aurion Biotech, Inc.",-,-,Inc,NaN
3,ROTR,"PHI Group, Inc.",-,-,Inc,NaN
4,ONE,One Power Company,-,-,Other,NaN
...,...,...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-,Inc,NaN
96,CHO,Chobani Inc.,-,-,Inc,NaN
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc,19.5
98,GLGX,"Gerson Lehrman Group, Inc.",-,-,Inc,NaN


In [13]:
withdrawn_ipos_df['Shares Offered'] = pd.to_numeric(withdrawn_ipos_df['Shares Offered'], errors='coerce')

In [14]:
withdrawn_ipos_df['Withdrawn Value'] = withdrawn_ipos_df['Shares Offered'] * withdrawn_ipos_df['Avg. Price']

In [15]:
withdrawn_ipos_df['Withdrawn Value'].notna().sum()

np.int64(71)

In [16]:
withdrawn_ipos_df.groupby('Company Class')['Withdrawn Value'].sum().sort_values(ascending=False)

Company Class
Corp        4.021000e+09
Inc         2.257164e+09
Other       7.679200e+08
Limited     5.497346e+08
Holdings    7.500000e+07
Group       3.378750e+07
Name: Withdrawn Value, dtype: float64

## Question 2. [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)

In [17]:
withdrawn_2024_df = get_withdrawn_ipos("https://stockanalysis.com/ipos/2024/")

In [18]:
withdrawn_2024_df.columns

Index(['IPO Date', 'Symbol', 'Company Name', 'IPO Price', 'Current', 'Return'], dtype='object')

In [19]:
withdrawn_2024_df['IPO Date'] = pd.to_datetime(withdrawn_2024_df['IPO Date'], errors='coerce')

withdrawn_2024_df.replace('-', np.nan, inplace=True)
withdrawn_2024_df = withdrawn_2024_df.dropna()

filtered_df = withdrawn_2024_df[withdrawn_2024_df['IPO Date'] < pd.Timestamp("2024-06-01")]

filtered_df = filtered_df.reset_index(drop=True)

In [20]:
filtered_df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2024-05-23,BOW,Bowhead Specialty Holdings Inc.,$17.00,$36.49,114.65%
1,2024-05-17,HDL,Super Hi International Holding Ltd.,$19.56,$18.70,-4.40%
2,2024-05-17,RFAI,RF Acquisition Corp II,$10.00,$10.60,6.00%
3,2024-05-15,JDZG,JIADE Limited,$4.00,$0.26,-93.40%
4,2024-05-15,RAY,Raytech Holding Limited,$4.00,$1.23,-69.25%
...,...,...,...,...,...,...
70,2024-01-18,CCTG,CCSC Technology International Holdings Limited,$6.00,$1.07,-82.17%
71,2024-01-18,PSBD,Palmer Square Capital BDC Inc.,$16.45,$14.50,-11.85%
72,2024-01-12,SYNX,Silynxcom Ltd.,$4.00,$2.07,-48.25%
73,2024-01-11,SDHC,Smith Douglas Homes Corp.,$21.00,$18.26,-13.05%


In [21]:
import time

def get_stock_features(ALL_TICKERS):
    stocks_df = pd.DataFrame() # Initialize an empty DataFrame

    for i, ticker in enumerate(ALL_TICKERS):

        ticker_obj = yf.Ticker(ticker)
        historyPrices = ticker_obj.history(period="max", interval="1d")

        historyPrices['Ticker'] = ticker

        for j in [1, 3, 7, 30, 90, 252, 365]:
            historyPrices['growth_' + str(j) + 'd'] = historyPrices['Close'] / historyPrices['Close'].shift(j)

        historyPrices['volatility'] = historyPrices['Close'].rolling(30).std() * np.sqrt(252)

        time.sleep(1)

        if stocks_df.empty:
            stocks_df = historyPrices
        else:
            stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=False)

    return stocks_df

In [22]:
full_stock_data_df = get_stock_features(filtered_df['Symbol'].unique().tolist())

In [23]:
full_stock_data_df.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,growth_1d,growth_3d,growth_7d,growth_30d,growth_90d,growth_252d,growth_365d,volatility
Date,,,,,,,,,,,,,,,,
2025-06-13 00:00:00-04:00,2.870,2.890,2.56,2.660,123100,0.0,0.0,ROMA,0.923611,0.960289,0.707447,1.934546,3.917526,4.666667,NaN,10.383358
2025-06-16 00:00:00-04:00,2.840,3.000,2.64,2.875,63100,0.0,0.0,ROMA,1.080827,0.958333,0.805322,1.955782,4.342900,5.424529,NaN,9.939668
2025-06-17 00:00:00-04:00,2.850,2.935,2.79,2.795,10200,0.0,0.0,ROMA,0.972174,0.970486,0.755405,1.814935,4.092240,5.008960,NaN,9.469241
2025-06-18 00:00:00-04:00,2.883,2.900,2.73,2.790,33600,0.0,0.0,ROMA,0.998211,1.048872,0.975524,1.516304,4.182909,5.157116,NaN,9.196931
2025-06-20 00:00:00-04:00,2.960,3.120,2.70,2.780,123300,0.0,0.0,ROMA,0.996416,0.966957,1.003610,1.390000,4.476651,5.325670,NaN,9.001998


In [24]:
full_stock_data_df['Sharpe'] = (full_stock_data_df['growth_252d'] - 0.045) / full_stock_data_df['volatility']

In [26]:
if not isinstance(full_stock_data_df.index, pd.DatetimeIndex):
    full_stock_data_df.index = pd.to_datetime(full_stock_data_df.index)

filtered_data_20250606 = full_stock_data_df.loc['2025-06-06']

In [28]:
filtered_data_20250606[['growth_252d', 'Sharpe']].describe()

,growth_252d,Sharpe
count,71.000000,71.000000
mean,1.152897,0.288285
std,1.406017,0.519028
min,0.024970,-0.079677
25%,0.293422,0.041215
50%,0.758065,0.083768
75%,1.362736,0.311507
max,8.097413,2.835668


In [29]:
top_10_by_growth = filtered_data_20250606.sort_values(by='growth_252d', ascending=False)['Ticker'].head(10).tolist()
print(f"Top 10 companies by growth_252d:\n{top_10_by_growth}\n")

Top 10 companies by growth_252d:
['JL', 'ROMA', 'UMAC', 'NNE', 'RBRK', 'AHR', 'AS', 'MRX', 'RDDT', 'MTEN']



In [30]:
top_10_by_sharpe = filtered_data_20250606.sort_values(by='Sharpe', ascending=False)['Ticker'].head(10).tolist()
print(f"Top 10 companies by Sharpe ratio:\n{top_10_by_sharpe}\n")

Top 10 companies by Sharpe ratio:
['BKHA', 'JVSA', 'LEGT', 'IBAC', 'HLXB', 'MNDR', 'DYCQ', 'INTJ', 'JL', 'TRSG']



In [31]:
common_companies = set(top_10_by_growth).intersection(set(top_10_by_sharpe))
print(f"There are {len(common_companies)} common companies in the top 10 lists:")
print(common_companies)

There are 1 common companies in the top 10 lists:
{'JL'}


## Question 3. [IPO] ‘Fixed Months Holding Strategy’

In [32]:
def get_stock_data_with_future_growth(ALL_TICKERS):
    stocks_df = pd.DataFrame()

    future_growth_days = [i * 21 for i in range(1, 13)]

    for i, ticker in enumerate(ALL_TICKERS):

        ticker_obj = yf.Ticker(ticker)
        historyPrices = ticker_obj.history(period="max", interval="1d")

        if historyPrices.empty:
            print(f"Warning: No historical data found for {ticker}. Skipping.")
            time.sleep(1)
            continue

        historyPrices['Ticker'] = ticker

        for m, days_shifted in enumerate(future_growth_days):
            col_name = f'future_growth_{m+1}m'
            historyPrices[col_name] = historyPrices['Close'].shift(-days_shifted) / historyPrices['Close']

        time.sleep(1)

        if stocks_df.empty:
            stocks_df = historyPrices
        else:
            stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=False)

    return stocks_df

In [33]:
ipo_full_stock_data_df = get_stock_data_with_future_growth(filtered_df['Symbol'].unique().tolist())

In [34]:
print(ipo_full_stock_data_df.head())
print(ipo_full_stock_data_df.columns)

                                Open       High        Low      Close  \
Date                                                                    
2024-05-23 00:00:00-04:00  23.000000  24.270000  22.139999  23.799999   
2024-05-24 00:00:00-04:00  24.260000  26.150000  23.980000  25.700001   
2024-05-28 00:00:00-04:00  25.850000  26.879999  25.075001  26.480000   
2024-05-29 00:00:00-04:00  26.440001  26.490000  25.500999  26.290001   
2024-05-30 00:00:00-04:00  27.209999  27.209999  25.500000  26.139999   

                            Volume  Dividends  Stock Splits Ticker  \
Date                                                                 
2024-05-23 00:00:00-04:00  3335800        0.0           0.0    BOW   
2024-05-24 00:00:00-04:00   990500        0.0           0.0    BOW   
2024-05-28 00:00:00-04:00   555100        0.0           0.0    BOW   
2024-05-29 00:00:00-04:00   302700        0.0           0.0    BOW   
2024-05-30 00:00:00-04:00   200900        0.0           0.0    BOW  

In [37]:
if not isinstance(ipo_full_stock_data_df.index, pd.DatetimeIndex):
    ipo_full_stock_data_df.index = pd.to_datetime(ipo_full_stock_data_df.index)

first_trading_days = ipo_full_stock_data_df.groupby('Ticker').apply(lambda x: x.index.min(), include_groups=False)
print("First Trading Day for each Ticker:")
print(first_trading_days.head())
print(f"\nTotal tickers: {len(first_trading_days)}")

First Trading Day for each Ticker:
Ticker
AHR    2024-02-07 00:00:00-05:00
ALAB   2024-03-20 00:00:00-04:00
ANRO   2024-02-02 00:00:00-05:00
AS     2024-02-01 00:00:00-05:00
AUNA   2024-03-22 00:00:00-04:00
dtype: datetime64[ns, America/New_York]

Total tickers: 75


In [39]:
ipo_full_stock_data_df_reset = ipo_full_stock_data_df.reset_index()
ipo_full_stock_data_df_reset.rename(columns={'index': 'Date'}, inplace=True)

In [40]:
first_trading_days_df = first_trading_days.reset_index()
first_trading_days_df.columns = ['Ticker', 'IPO_Date']

In [42]:
ipo_full_stock_data_df_reset['Date'] = pd.to_datetime(ipo_full_stock_data_df_reset['Date'])
first_trading_days_df['IPO_Date'] = pd.to_datetime(first_trading_days_df['IPO_Date'])

In [44]:
merged_data = pd.merge(
    ipo_full_stock_data_df_reset,
    first_trading_days_df,
    how='inner',
    left_on=['Ticker', 'Date'],
    right_on=['Ticker', 'IPO_Date']
)

merged_data.drop(columns=['IPO_Date'], inplace=True)

In [47]:
print(f"\nShape of the resulting DataFrame: {merged_data.shape}")


Shape of the resulting DataFrame: (75, 21)


In [49]:
future_growth_columns = [f'future_growth_{i}m' for i in range(1, 13)]
merged_data[future_growth_columns].describe().T

,count,mean,std,min,25%,50%,75%,max
future_growth_1m,75.0,0.927259,0.346261,0.098947,0.778984,0.977000,1.046509,2.646505
future_growth_2m,75.0,0.940544,0.574545,0.073800,0.685815,1.000000,1.154013,4.874759
future_growth_3m,75.0,0.833824,0.409763,0.060947,0.511212,0.927500,1.069085,2.040000
future_growth_4m,75.0,0.825086,0.401969,0.045368,0.517233,0.909091,1.134300,1.605000
future_growth_5m,75.0,0.803769,0.488349,0.054109,0.448403,0.821092,1.016381,3.213873
future_growth_6m,75.0,0.864186,0.653180,0.061432,0.384560,0.802239,1.093948,3.670520
future_growth_7m,75.0,0.847150,0.712944,0.044086,0.296870,0.844875,1.114468,5.122350
future_growth_8m,75.0,0.832982,0.762422,0.043103,0.208677,0.812109,1.082365,5.171484
future_growth_9m,75.0,0.881777,0.936956,0.033144,0.226740,0.822715,1.049719,6.764933
future_growth_10m,74.0,0.917943,0.911431,0.037769,0.242424,0.772592,1.200678,5.352601


## Question 4. [Strategy] Simple RSI-Based Trading Strategy

In [50]:
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

In [52]:
ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS

In [ ]:
stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  ticker_obj = yf.Ticker(ticker)

  # historyPrices = yf.download(tickers = ticker,
  #                    period = "max",
  #                    interval = "1d")
  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

In [ ]:
def get_ticker_type(ticker:str, us_stocks_list, eu_stocks_list, india_stocks_list):
  if ticker in us_stocks_list:
    return 'US'
  elif ticker in eu_stocks_list:
    return 'EU'
  elif ticker in india_stocks_list:
    return 'INDIA'
  else:
    return 'ERROR'

In [ ]:
stocks_df['ticker_type'] = stocks_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS))

In [ ]:
stocks_df.ticker_type.value_counts()

In [ ]:
stocks_df.Ticker.nunique()

In [ ]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'],utc=True)
stocks_df['Volume'] = stocks_df['Volume']*1.0

# https://stackoverflow.com/questions/51712269/how-to-run-ta-lib-on-multiple-columns-of-a-pandas-dataframe
for f in ['Open','High','Low','Close', 'Volume', 'Close']:
  stocks_df.loc[:,f] = stocks_df.loc[:,f].astype('float64')

In [ ]:
import talib

url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
!pip install conda-package-handling
!wget https://anaconda.org/conda-forge/ta-lib/0.5.1/download/linux-64/ta-lib-0.5.1-py311h9ecbd09_0.conda
!cph x ta-lib-0.5.1-py311h9ecbd09_0.conda

!rm -rf /usr/local/lib/python3.11/dist-packages/talib

!mv ./ta-lib-0.5.1-py311h9ecbd09_0/lib/python3.11/site-packages/talib /usr/local/lib/python3.11/dist-packages/

In [ ]:
stocks_df = stocks_df.sort_values(by=['Ticker', 'Date']).reset_index(drop=True)

print("Calculating RSI for all tickers...")
stocks_df['rsi'] = stocks_df.groupby('Ticker')['Close'].transform(
    lambda x: talib.RSI(x.values, timeperiod=14)
)

In [ ]:
print(stocks_df['rsi'].dtype)

In [ ]:
len(stocks_df)

In [ ]:
rsi_threshold = 25

selected_df = stocks_df[
    (stocks_df['rsi'] < rsi_threshold) &
    (stocks_df['Date'] >= '2000-01-01') &
    (stocks_df['Date'] <= '2025-06-01')
]

In [ ]:
len(selected_df)

In [ ]:
net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()

In [ ]:
print(f"Total profit earned : {net_income:,.2f}")

In [ ]:
rsi_threshold = 25

selected_df = stocks_df[
    (stocks_df['rsi'] < rsi_threshold) &
    (stocks_df['Date'] >= '2000-01-01') &
    (stocks_df['Date'] <= '2025-06-01') &
    (stocks_df['growth_future_30d'].notna())
].copy()

selected_df['growth_future_30d'] = selected_df['growth_future_30d'].astype(float)

net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()

net_income_k = net_income / 1000

print(f"Total profit earned: ${net_income_k:,.2f}K")

In [ ]:
import gdown

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")